## Imports

In [1]:
from collections import Counter, defaultdict
from functools import reduce
from itertools import combinations, chain, product
from math import floor, ceil, sin, cos, radians
from operator import itemgetter

import re

## Day 1

In [2]:
with open('data/day01.txt', 'r+') as f:
    expense_report = [int(s) for s in f.read().split('\n') if s != '']

In [3]:
def expense_product(expense_report, n):
    ints = [c for c in combinations(expense_report, n) if sum(c) == 2020][0]
    return reduce((lambda x, y: x * y), ints)

* **Part 1**

In [4]:
expense_product(expense_report, 2)

1005459

* **Part 2**

In [5]:
expense_product(expense_report, 3)

92643264

## Day 2

In [6]:
with open('data/day02.txt', 'r+') as f:
    passwords_db = [s for s in f.read().split('\n') if s != '']

In [7]:
def count_valid_passwords(db, func):
    return sum([func(password) for password in db])

* **Part 1**

In [8]:
def is_valid_v1(password):
    rule, pwd = password.split(':')
    min_expected, max_expected = [int(x) for x in re.findall('\d+', rule)]
    letter = re.findall('[a-z]', rule)[0]
    actual = Counter(pwd.strip())[letter]
    return actual >= min_expected and actual < max_expected + 1

In [9]:
count_valid_passwords(passwords_db, is_valid_v1)

456

* **Part 2**

In [10]:
def is_valid_v2(password):
    rule, pwd = [s.strip() for s in password.split(':')]
    ix_1, ix_2 = [int(x) - 1 for x in re.findall('\d+', rule)]
    letter = re.findall('[a-z]', rule)[0]
    return (pwd[ix_1] == letter or pwd[ix_2] == letter) and not (pwd[ix_1] == letter and pwd[ix_2] == letter)

In [11]:
count_valid_passwords(passwords_db, is_valid_v2)

308

## Day 3

In [12]:
with open('data/day03.txt', 'r+') as f:
    tree_map = [list(s) for s in f.read().split('\n') if s != '']

In [13]:
def check_slope(tree_map, r, d):
    return sum(['#' == tree_map[i][j % len(tree_map[0])] 
                for i, j in zip(range(0,len(tree_map), d), range(0, len(tree_map) * r , r))])

* **Part 1**

In [14]:
check_slope(tree_map, 3, 1)

169

* **Part**

In [15]:
trees_encountered = [check_slope(tree_map, r, d) for r, d in [(1, 1), (3, 1), (5, 1), (7, 1), (1, 2)]]

In [16]:
reduce((lambda x, y: x * y), trees_encountered)

7560370818

## Day 4

In [17]:
with open("data/day04.txt") as f:
    lines = [line.rstrip('\n') for line in f]

In [18]:
def get_fields(doc):
    fields = {}
    for s in doc.split():
        k, v = s.split(':')
        fields[k] = v
    return(fields)

def get_documents(lines):
    empty_lines_index = [i for i, s in enumerate(lines) if s == ''] + [len(lines)]
    doc_strings = [' '.join(lines[ix[0]: ix[1]]) for ix in zip([0] + [i + 1 for i in empty_lines_index], empty_lines_index)]
    return [get_fields(doc) for doc in doc_strings]

* **Part 1**

In [19]:
def is_valid_document_v1(doc):
    return all([field in doc for field in ['byr', 'iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid']])

In [20]:
sum([is_valid_document_v1(doc) for doc in get_documents(lines)])

226

* **Part 2**

In [21]:
def is_valid_byr(byr):
    return int(byr) >= 1920 and int(byr) <= 2002

def is_valid_iyr(iyr):
    return int(iyr) >= 2010 and int(iyr) <= 2020

def is_valid_eyr(eyr):
    return int(eyr) >= 2020 and int(eyr) <= 2030

def is_valid_hgt(hgt):
    results = re.findall('(\d+)(cm|in)', hgt)
    if results:
        n, unit = results[0]
        if unit == 'cm':
            return int(n) >= 150 and int(n) <= 193
        elif unit == 'in':
            return int(n) >= 59 and int(n) <= 76
    return False

def is_valid_hcl(hcl):
    return len(re.findall('#[aA-zZ0-9]{4}', hcl)) > 0

def is_valid_ecl(ecl):
    return ecl in ['amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth']

def is_valid_pid(pid):
    return len(re.findall('\d', pid)) == len(pid)

In [22]:
def is_valid_document_v2(doc):
    if is_valid_document_v1(doc):
        return is_valid_byr(doc['byr']) and is_valid_iyr(doc['iyr'])\
            and is_valid_eyr(doc['eyr']) and is_valid_hgt(doc['hgt'])\
            and is_valid_hcl(doc['hcl']) and is_valid_ecl(doc['ecl'])\
            and is_valid_pid(doc['pid'])
    return False

In [23]:
sum([is_valid_document_v2(doc) for doc in get_documents(lines)])

162

## Day 5

In [24]:
with open("data/day05.txt") as f:
    boarding_passes = [line.rstrip('\n') for line in f]

In [25]:
def keep_lower_half(pair):
    return (pair[0], floor((pair[1] - pair[0]) / 2) + pair[0])

def keep_upper_half(pair):
    return (ceil((pair[1] - pair[0]) / 2) + pair[0], pair[1])

* **Part 1**

In [26]:
def get_row_and_col_number(boarding_pass):
    rows = (0, 127)
    cols = (0, 7)

    for char in boarding_pass:
        if char == 'F':
            rows = keep_lower_half(rows)
        elif char == 'B':
            rows = keep_upper_half(rows)
        elif char == 'L':
            cols = keep_lower_half(cols)
        elif char == 'R':
            cols = keep_upper_half(cols)
    
    if rows[0] == rows[1] and cols[0] == cols[1]:
        return rows[0], cols[0]

In [27]:
def get_seat_id(boarding_pass):
    row, col = get_row_and_col_number(boarding_pass)
    return row * 8 + col

In [28]:
max([get_seat_id(bp) for bp in boarding_passes])

965

* **Part 2**

In [29]:
ids = sorted([get_seat_id(bp) for bp in boarding_passes])

In [30]:
[(i, j, j - i) for i, j in zip(ids[:-1], ids[1:]) if j - i > 1]

[(523, 525, 2)]

## Day 6

In [31]:
with open('data/day06.txt', 'r+') as f:
    groups = f.read().split('\n\n')

* **Part 1**

In [32]:
sum([len(set(g.replace('\n', ''))) for g in groups])

6630

* **Part 2**

In [33]:
def count_group_answers(group):
    n_people = len([a for a in group.split('\n') if a != ''])
    return sum([1 for c in Counter(group.replace('\n', '')).values() if c == n_people])

In [34]:
sum([count_group_answers(g) for g in groups])

3437

## Day 7

In [35]:
with open('data/day07.txt', 'r+') as f:
    rules = [s for s in f.read().split('\n') if s != '']

In [36]:
def contained_in(color):
    '''returns the list of colored bags that can contain a bag of color = color'''
    return [r.split('bag')[0].strip() for r in rules if re.findall(f'(?<!^){color} bag', r)]

* **Part 1**

In [37]:
colors = contained_in('shiny gold')

for color in colors:
    colors += contained_in(color)

In [38]:
len(set(colors))

272

## Day 9

In [39]:
with open('data/day09.txt', 'r+') as f:
    numbers = [int(s) for s in f.read().split('\n') if s != '']

* **Part 1**

In [40]:
def first_invalid(numbers, preamble_size=25):
    for ix in range(preamble_size, len(numbers)):
        if not any([i + j == numbers[ix] for i, j in combinations(numbers[(ix - preamble_size):ix], 2)]):
            return numbers[ix]

In [41]:
first_invalid(numbers)

32321523

* **Part 2**

In [42]:
test = [35, 20, 15, 25, 47, 40, 62, 55, 65, 95, 102, 117, 150, 182, 127, 219, 299, 277, 309, 576]

In [43]:
for n in range(2, len(test)):
    for subset in zip(*[test[i:] for i in range(n)]):
        if sum(subset) == 127:
            print(subset)

(15, 25, 47, 40)


In [44]:
def find_encryption_weakness(numbers, preamble_size=25):
    invalid_n = first_invalid(numbers, preamble_size)
    for n in range(2, len(numbers)):
        for subset in zip(*[numbers[i:] for i in range(n)]):
            if sum(subset) == invalid_n:
                return min(subset) + max(subset)

In [45]:
find_encryption_weakness(test, 5)

62

In [46]:
find_encryption_weakness(numbers, 25)

4794981

## Day 10

In [47]:
with open('data/day10.txt', 'r+') as f:
    adapters = [int(s) for s in f.read().split('\n') if s != '']

In [48]:
def find_all_differences(adapters):
    adapters = sorted(adapters)
    return [adapters[0] - 0] + [j-i for i, j in zip(adapters[:-1], adapters[1:])] + [3]

In [49]:
def distribution(adapters):
    diffs = find_all_differences(adapters)
    return sum([1 for d in diffs if d == 1]) * sum([1 for d in diffs if d == 3])

In [50]:
distribution(adapters)

3034

## Day 11

In [51]:
with open('data/day11.txt', 'r+') as f:
    seat_map = [list(s) for s in f.read().split('\n') if s != '']

In [52]:
def get_neighbors(x, y, seat_map):
    return [seat_map[x + i][y + j] 
            for i in [-1, 0, 1] 
            for j in [-1, 0, 1] 
            if (i != 0 or j != 0) 
            and x + i in range(len(seat_map))
            and y + j in range(len(seat_map[0]))]

In [53]:
def count_occupied(x, y, seat_map):
    return sum([neighbor == '#' for neighbor in get_neighbors(x, y, seat_map)])

In [54]:
def update_map(seat_map):
    new_map = [x[:] for x in seat_map]
    for x in range(len(seat_map)):
        for y in range(len(seat_map[x])):
            if seat_map[x][y] == 'L' and count_occupied(x, y, seat_map) == 0:
                new_map[x][y] = '#'
            elif seat_map[x][y] == '#' and count_occupied(x, y, seat_map) >= 4:
                new_map[x][y] = 'L'
    return new_map
                    

In [55]:
def total_occupied_seats(seat_map):
    current_map = seat_map
    new_map = update_map(current_map)

    while current_map != new_map:
        current_map = new_map
        new_map = update_map(current_map)
    return sum([seat == '#' for row in new_map for seat in row])

In [56]:
total_occupied_seats(seat_map)

2270

## Day 12

In [57]:
with open('data/day12.txt', 'r+') as f:
    directions = [re.findall('(\w)(\d+)', s)[0] for s in f.read().split('\n') if s != '']

* **Part 1**

In [58]:
D = ['W', 'N', 'E', 'S']

def change_direction(current_facing_direction, direction, degrees):
    if direction == 'L':
        return D[D.index(current_facing_direction) - round(int(degrees)/ 90)]
    if direction == 'R':
        return D[(D.index(current_facing_direction) + round(int(degrees)/ 90)) % 4]

In [59]:
def manhattan_dist(directions):
    x, y = 0, 0
    current_direction = 'E'

    for direction, value in directions:
        direction = direction.replace('F', current_direction)
        if direction == 'E':
            x += int(value)
        if direction == 'W':
            x -= int(value)
        if direction == 'N':
            y += int(value)
        if direction == 'S':
            y -= int(value)
        if direction in ['R', 'L']:
            current_direction = change_direction(current_direction, direction, int(value))
    return abs(x) + abs(y)

In [60]:
test = '''F10
N3
F7
R90
F11'''
test = [re.findall('(\w)(\d+)', s)[0] for s in test.split('\n') if s != '']

In [61]:
manhattan_dist(test)

25

In [62]:
manhattan_dist(directions)

415

* **Part 2**

In [63]:
def rotate_coordinates(wx, wy, direction, degrees):
    degrees = degrees if direction == 'R' else 360 - degrees 
    sin_angle = round(sin(radians(degrees)))
    cos_angle = round(cos(radians(degrees)))
    return (wx * cos_angle) + (wy * sin_angle), - (wx * sin_angle) + (wy * cos_angle)

In [64]:
def manhattan_dist_with_waypoint(directions):
    x, y = 0, 0
    wx, wy = 10, 1

    for direction, value in directions:
        if direction == 'N':
            wy += int(value)
        if direction == 'S':
            wy -= int(value)
        if direction == 'E':
            wx += int(value)
        if direction == 'W':
            wx -= int(value)
        if direction == 'F':
            x += int(value) * wx
            y += int(value) * wy
        if direction in ['R', 'L']:
            wx, wy = rotate_coordinates(wx, wy, direction, int(value))
    return abs(x) + abs(y)

In [65]:
manhattan_dist_with_waypoint(test)

286

In [66]:
manhattan_dist_with_waypoint(directions)

29401

## Day 13

* **Part 1**

In [166]:
with open('data/day13.txt', 'r+') as f:
    line1, line2 = f.read().strip('\n').split('\n')
    timestamp = int(line1)
    buses = [int(bus) for bus in re.findall('(\d+)', line2)]

In [167]:
def departure_time(timestamp, buses):
    departure_times = [ceil(timestamp / busid) * busid for busid in buses]
    ix, dep_time = min(enumerate(departure_times), key=itemgetter(1))
    return (dep_time - timestamp) * buses[ix]

In [168]:
departure_time(timestamp, buses)

2238

## Day 14

In [20]:
with open('data/day14.txt', 'r+') as f:
    lines = [s for s in f.read().rsplit('\n') if s != '']

In [21]:
def memory_instruction(string):
    mem, n = string.split(' = ')
    return mem, list('{0:036b}'.format(int(n)))

def apply_bitmask(mask, binary):
    return ''.join([r if m == 'X' else m for r, m in zip(binary, mask)])

In [22]:
def sum_of_values_in_memory(lines):
    
    memory = {}
    mask = ''
    mask_values = []
    
    for line in lines:
        if 'mask' in line:
            mask = line.split(' = ')[1]
        else:
            if line != '':
                mem, b = memory_instruction(line)
                memory[mem] = apply_bitmask(mask, b)
    return sum([int(b, 2) for mem, b in memory.items()])

In [23]:
test = '''mask = XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X
mem[8] = 11
mem[7] = 101
mem[8] = 0
'''

In [24]:
sum_of_values_in_memory(test.split('\n'))

165

In [25]:
sum_of_values_in_memory(lines)

9879607673316

* **Part 2**

In [26]:
def get_memory_slot(line):
    mem, value = line.split(' = ')
    mem = int(re.findall('\d+', mem)[0])
    value = int(value)
    return '{0:036b}'.format(mem), value

In [27]:
def apply_bitmask(mask, binary):
    out = []
    for r, m in zip(binary, mask):
        if m == '1':
            out.append('1')
        elif m == 'X':
            out.append(m)
        else:
            out.append(r)
    return ''.join(out)

In [28]:
memory = {}
mask = ''
for line in lines:
    if 'mask' in line:
        mask = line.split(' = ')[1]
    else:
        if line != '':
            mem, v = get_memory_slot(line)
            result = apply_bitmask(mask, mem)
            for perm in product(['0', '1'], repeat=len(re.findall('X', result))):
                mem = list(result)
                for i in perm:
                    mem[mem.index('X')] = i                    
                memory[int(''.join(mem), 2)] = v

In [29]:
sum([n for mem, n in memory.items()])

3435342392262

## Day 15

* **Part 1**

In [158]:
starting_numbers = [1,20,11,6,12,0]
seen_at = defaultdict(list)

for i, n in enumerate(starting_numbers):
    seen_at[n].append(i + 1)

last_n = starting_numbers[-1]

for turn in range(len(starting_numbers) + 1, 2021):
    if last_n in seen_at and len(seen_at[last_n]) > 1:
        last_n = seen_at[last_n][-1] - seen_at[last_n][-2]
    else:
        last_n = 0
    seen_at[last_n].append(turn)
print('turn', turn, 'n', last_n)

turn 2020 n 1085


* **Part 2**

In [160]:
starting_numbers = [1,20,11,6,12,0]
seen_at = defaultdict(list)

for i, n in enumerate(starting_numbers):
    seen_at[n].append(i + 1)

last_n = starting_numbers[-1]

for turn in range(len(starting_numbers) + 1, 30000001):
    if last_n in seen_at and len(seen_at[last_n]) > 1:
        last_n = seen_at[last_n][-1] - seen_at[last_n][-2]
    else:
        last_n = 0
    seen_at[last_n].append(turn)
print('turn', turn, 'n', last_n)

turn 30000000 n 10652


## Day 16

In [2]:
with open('data/day16.txt', 'r+') as f:
    raw = f.read()

* **Part 1**

In [3]:
def extract_intervals(s):
    ints = [int(i) for i in re.findall('\d+', s)]
    return tuple(ints[:2]), tuple(ints[2:])

In [4]:
rules = [extract_intervals(s) for s in raw.split('your ticket')[0].split('\n') if s != '']

In [5]:
nearby_tickets = [s for s in raw.split('nearby tickets:\n')[1].split('\n') if s != '']

In [6]:
def is_valid(n, r):
    return ((n >= r[0][0] and n <= r[0][1]) or (n >= r[1][0] and n <= r[1][1]))

In [7]:
invalid = []
for t in nearby_tickets:
    t_ = [int(n) for n in t.split(',')]
    for n in t_:
        if not any([is_valid(n, r) for r in rules]):
            invalid.append(n)

In [8]:
sum(invalid)

21071

* **Part 2**

In [9]:
def is_invalid_ticket(t):
    t_ = [int(n) for n in t.split(',')]
    for n in t_:
        if not any([is_valid(n, r) for r in rules]):
            return True
    return False

In [10]:
valid_tickets = [[int(n) for n in t.split(',')] for t in nearby_tickets if not is_invalid_ticket(t)]

In [11]:
len(valid_tickets)

190

In [12]:
def extract_rules(raw):
    rules = {}
    
    lines = [s for s in raw.split('your ticket')[0].split('\n') if s != '']
    for line in lines:
        name, intervals = line.split(':')
        rules[name] = extract_intervals(intervals)
    return rules

In [13]:
rules = extract_rules(raw)
rules

{'departure location': ((45, 422), (444, 950)),
 'departure station': ((36, 741), (752, 956)),
 'departure platform': ((46, 788), (806, 967)),
 'departure track': ((46, 57), (70, 950)),
 'departure date': ((35, 99), (108, 974)),
 'departure time': ((42, 883), (903, 962)),
 'arrival location': ((47, 83), (95, 953)),
 'arrival station': ((31, 227), (240, 970)),
 'arrival platform': ((48, 840), (853, 964)),
 'arrival track': ((49, 487), (499, 964)),
 'class': ((33, 363), (381, 959)),
 'duration': ((35, 509), (522, 951)),
 'price': ((38, 590), (601, 950)),
 'route': ((41, 266), (285, 962)),
 'row': ((44, 402), (419, 962)),
 'seat': ((41, 615), (634, 956)),
 'train': ((47, 156), (178, 954)),
 'type': ((44, 313), (338, 964)),
 'wagon': ((30, 110), (133, 970)),
 'zone': ((38, 541), (550, 965))}

In [15]:
your_ticket = [int(n) for n in raw.split('your ticket:\n')[1].split('\n')[0].split(',')]

In [20]:
print(your_ticket)

[109, 199, 223, 179, 97, 227, 197, 151, 73, 79, 211, 181, 71, 139, 53, 149, 137, 191, 83, 193]


In [16]:
# reduce((lambda x, y: x * y), [your_ticket[i] for i, name in enumerate(rules_order) if 'departure' in name])

19064923187293

In [23]:
len(rules)

20

In [37]:
valid_rules = defaultdict(list)
for i in range(len(rules)):
    for name, intervals in rules.items():
        if all([is_valid(t[i], intervals) for t in valid_tickets]):
            valid_rules[i].append(name)

In [38]:
valid_rules

defaultdict(list,
            {0: ['departure location',
              'departure station',
              'departure platform',
              'departure track',
              'departure date',
              'departure time',
              'arrival station',
              'arrival platform',
              'arrival track',
              'class',
              'duration',
              'price',
              'route',
              'row',
              'seat',
              'train',
              'type',
              'wagon',
              'zone'],
             1: ['arrival track', 'duration', 'price', 'row'],
             2: ['departure location',
              'departure station',
              'departure platform',
              'departure track',
              'departure date',
              'departure time',
              'arrival track',
              'duration',
              'price',
              'row'],
             3: ['departure location',
              'departure station',
  